In [1]:
# !pip install matplotlib==3.3.0

In [2]:
import tensorboard

In [3]:
tensorboard.__version__

'2.6.0'

In [4]:
import matplotlib.pyplot as plt
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
import torch, torchvision
import detectron2
from detectron2.utils.logger import setup_logger

setup_logger()
import numpy as np
import pandas as pd
import wandb
import os, json, cv2, random
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from tqdm import tqdm
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [5]:
# Version
print(torch.__version__, torchvision.__version__)
data = pd.read_csv("./Data.csv")
idx = 0

1.9.1+cu111 0.10.1+cu111


In [6]:
# data['XMin'] = data['XMin']*255
# data['YMin'] = data['YMin']*255
# data['XMax'] = data['XMax']*255
# data['YMax'] = data['YMax']*255

In [7]:
# for i in tqdm(range(len(data))):
#     info = data.iloc[i]
# #     ymin,xmin,ymax,xmax = info['XMin'],info['YMin'],info['XMax'],info['YMax']
#     height,width = cv2.imread(f'./Img/{info["Path"]}').shape[:2]
#     xmin,ymin,xmax,ymax = info['XMin'],info['YMin'],info['XMax'],info['YMax']
#     print(xmin,ymin,xmax,ymax)
#     xmin = round(xmin * width)
#     xmax = round(xmax * width)
#     ymin = round(ymin * height)
#     ymax = round(ymax * height)
#     x = xmin
#     y = ymin
#     w = xmax - xmin
#     h = ymax - ymin
# #     x,y,w,h = x*255,y*255,w*255,h*255
#     x,y,w,h = round(x),round(y),round(w),round(h)
#     print(x,y,w,h,xmin,xmax,ymin,ymax,height,width)
#     img = cv2.imread(f'./Img/{info["Path"]}')
#     cv2.imwrite('./output.png',img)
#     roi=img[y:y+h,x:x+w]
#     cv2.imwrite(str('crop') + '.jpg', roi)
#     cv2.rectangle(img,(x,y),(x+w,y+h),(200,0,0),10)
#     cv2.imwrite(str('box') + '.jpg', img)
#     plt.imshow(img)

In [8]:
# height

In [9]:
# width

In [10]:
# xmax - xmin

In [11]:
# img.shape

In [12]:
# info = data.iloc[50]
# # xmin,ymin,xmax,ymax = info['XMin']*im[0],info['YMin']*im[1],info['XMax']*im[0],info['YMax']*im[1]
# xmin,ymin,xmax,ymax = info['XMin']*255,info['YMin']*255,info['XMax']*255,info['YMax']*255
# # xmin,ymin,xmax,ymax = info['XMin'],info['YMin'],info['XMax'],info['YMax']
# x = xmin
# y = ymin
# w = xmax - xmin
# h = ymax - ymin
# # x,y,w,h = x*255,y*255,w*255,h*255
# # x,y,w,h = x*255,y*255,w*255,h*255
# x,y,w,h = round(x),round(y),round(w),round(h)

In [13]:
# Loading Data
def load_data(data=data, test=False):
    if test is True:
        if "data.npy" in os.listdir("./"):
            data = np.load("./data.npy", allow_pickle=True)[:500]
            return data
    if "data.npy" in os.listdir("./"):
        data = np.load("./data.npy", allow_pickle=True)
        return data
    new_data = []
    for idx in tqdm(range(len(data))):
        record = {}
        info = data.iloc[idx]
        height, width = cv2.imread("./Img/" + info["Path"]).shape[:2]
        xmin, ymin, xmax, ymax = info["XMin"], info["YMin"], info["XMax"], info["YMax"]
        xmin = round(xmin * width)
        xmax = round(xmax * width)
        ymin = round(ymin * height)
        ymax = round(ymax * height)
        record["file_name"] = "./Img/" + info["Path"]
        record["height"] = height
        record["width"] = width
        record["cateogry_id"] = 0
        objs = [
            {
                "bbox": [xmin,ymin,xmax,ymax],
                "bbox_mode": BoxMode.XYXY_ABS,
                "category_id": 0,
            }
        ]
        record["image_id"] = idx
        record["annotations"] = objs
        new_data.append(record)
    np.save("data.npy", new_data)
    return new_data

In [14]:
# Config
labels = ["Card"]

In [15]:
# Adding the data
DatasetCatalog.register("data", lambda: load_data())
MetadataCatalog.get("data").set(thing_classes=labels)
metadata = MetadataCatalog.get("data")
DatasetCatalog.register("test", lambda: load_data(test=True))
MetadataCatalog.get("test").set(thing_classes=labels)
metadata_test = MetadataCatalog.get("test")

In [ ]:
models = [
#     "fast_rcnn_R_50_FPN_1x.yaml",
    "faster_rcnn_R_50_C4_1x.yaml",
    "faster_rcnn_R_50_C4_3x.yaml",
    "faster_rcnn_R_50_DC5_1x.yaml",
    "faster_rcnn_R_50_DC5_3x.yaml",
    "retinanet_R_50_FPN_1x.py",
    "retinanet_R_50_FPN_1x.yaml",
    "retinanet_R_50_FPN_3x.yaml",
    "rpn_R_50_C4_1x.yaml",
    "rpn_R_50_FPN_1x.yaml"
    "faster_rcnn_R_50_FPN_1x.yaml",
    "faster_rcnn_R_50_FPN_3x.yaml",
    "faster_rcnn_R_101_DC5_3x.yaml",
    "faster_rcnn_R_101_FPN_3x.yaml",
    "faster_rcnn_X_101_32x8d_FPN_3x.yaml",
    "retinanet_R_101_FPN_3x.yaml",

]
for model in models:
    try:
        model = f"COCO-Detection/{model}"
        torch.cuda.empty_cache()
        wandb.init(sync_tensorboard=True,name=f'{model}')
        torch.cuda.empty_cache()
        cfg = get_cfg()
        torch.cuda.empty_cache()
        cfg.merge_from_file(model_zoo.get_config_file(model))
        torch.cuda.empty_cache()
        cfg.DATASETS.TRAIN = ('data',)
        torch.cuda.empty_cache()
        cfg.DATASETS.TEST = ()
        torch.cuda.empty_cache()
        cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model)
        torch.cuda.empty_cache()
        cfg.SOLVER.MAX_ITER = 1000
        torch.cuda.empty_cache()
        # cfg.TEST.EVAL_PERIOD = 50
        cfg.SOLVER.BASE_LR = 0.00025
        torch.cuda.empty_cache()
        cfg.SOLVER.STEPS = []
        torch.cuda.empty_cache()
        cfg.SOLVER.IMS_PER_BATCH = (1)
        torch.cuda.empty_cache()
        cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(labels)
        torch.cuda.empty_cache()
        cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
        torch.cuda.empty_cache()
        trainer = DefaultTrainer(cfg)
        torch.cuda.empty_cache()
        trainer.resume_or_load(resume=False)
        torch.cuda.empty_cache()
        trainer.train()
        torch.cuda.empty_cache()
        cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.475
        torch.cuda.empty_cache()
        cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
        torch.cuda.empty_cache()
        predictor = DefaultPredictor(cfg)
        torch.cuda.empty_cache()
        evaluator = COCOEvaluator("test", output_dir="./output")
        torch.cuda.empty_cache()
        val_loader = build_detection_test_loader(cfg,"test")
        torch.cuda.empty_cache()
        metrics = inference_on_dataset(predictor.model, val_loader, evaluator)
        torch.cuda.empty_cache()
        wandb.log(metrics)
        torch.cuda.empty_cache()
        for img in os.listdir("./test_imgs/"):
            torch.cuda.empty_cache()
            v = Visualizer(cv2.imread(f"./test_imgs/{img}")[:, :, ::-1], metadata=metadata)
            torch.cuda.empty_cache()
            v = v.draw_instance_predictions(
                predictor(cv2.imread(f"./test_imgs/{img}"))["instances"].to("cpu")
            )
            torch.cuda.empty_cache()
            v = v.get_image()[:, :, ::-1]
            torch.cuda.empty_cache()
            plt.figure(figsize=(12, 6))
            torch.cuda.empty_cache()
            plt.imshow(v)
            torch.cuda.empty_cache()
            plt.savefig("./img.png")
            torch.cuda.empty_cache()
            plt.close()
            torch.cuda.empty_cache()
        wandb.finish()
    except:
        torch.cuda.empty_cache()

2021-10-14 00:14:16.059920: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.0
wandb: Run data is saved locally in wandb/run-20211014_001418-y2yvawo3
wandb: Syncing run COCO-Detection/faster_rcnn_R_50_C4_1x.yaml



[10/14 00:14:34 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)


Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 2048) in the checkpoint but (2, 2048) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 2048) in the checkpoint but (4, 2048) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, we

[10/14 00:14:35 d2.engine.train_loop]: Starting training from iteration 0
[10/14 00:14:39 d2.utils.events]:  eta: 0:03:03  iter: 19  total_loss: 1.11  loss_cls: 0.6807  loss_box_reg: 0.0003511  loss_rpn_cls: 0.3705  loss_rpn_loc: 0.09109  time: 0.1948  data_time: 0.0229  lr: 4.9953e-06  max_mem: 1254M
[10/14 00:14:43 d2.utils.events]:  eta: 0:02:55  iter: 39  total_loss: 1.021  loss_cls: 0.6036  loss_box_reg: 0.0003399  loss_rpn_cls: 0.3395  loss_rpn_loc: 0.09647  time: 0.1882  data_time: 0.0013  lr: 9.9902e-06  max_mem: 1254M
[10/14 00:14:47 d2.utils.events]:  eta: 0:02:50  iter: 59  total_loss: 0.8657  loss_cls: 0.4723  loss_box_reg: 0.0003396  loss_rpn_cls: 0.347  loss_rpn_loc: 0.0875  time: 0.1847  data_time: 0.0014  lr: 1.4985e-05  max_mem: 1272M
[10/14 00:14:50 d2.utils.events]:  eta: 0:02:45  iter: 79  total_loss: 0.8459  loss_cls: 0.3801  loss_box_reg: 0.0002072  loss_rpn_cls: 0.3485  loss_rpn_loc: 0.1018  time: 0.1820  data_time: 0.0012  lr: 1.998e-05  max_mem: 1280M
[10/14 00

wandb: Waiting for W&B process to finish, PID 86045
wandb: Program ended successfully.


...

wandb:                                                                                
wandb: Find user logs for this run at: wandb/run-20211014_002523-3jdxd23g/logs/debug.log
wandb: Find internal logs for this run at: wandb/run-20211014_002523-3jdxd23g/logs/debug-internal.log
wandb: Run summary:
wandb:        _step 0
wandb:     _runtime 308
wandb:   _timestamp 1634151633
wandb: Run history:
wandb:        _step ▁
wandb:     _runtime ▁
wandb:   _timestamp ▁
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: 
wandb: Synced COCO-Detection/faster_rcnn_R_50_DC5_1x.yaml: https://app.wandb.ai/ranuga-d/Object-Detection/runs/3jdxd23g
Ignoring settings passed to wandb.setup() which has already been configured.
wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.0
wandb: Run data is saved locally in wandb/run-20211014_003046-3qhmmr9v
wandb: Syncing run COCO-Det


[10/14 00:30:50 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)


Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, we

[10/14 00:30:53 d2.engine.train_loop]: Starting training from iteration 0
[10/14 00:30:58 d2.utils.events]:  eta: 0:03:45  iter: 19  total_loss: 1.094  loss_cls: 0.7077  loss_box_reg: 0.0003688  loss_rpn_cls: 0.2386  loss_rpn_loc: 0.08245  time: 0.2296  data_time: 0.0264  lr: 4.9953e-06  max_mem: 3807M
[10/14 00:31:03 d2.utils.events]:  eta: 0:03:45  iter: 39  total_loss: 0.9404  loss_cls: 0.6017  loss_box_reg: 0.0002847  loss_rpn_cls: 0.2882  loss_rpn_loc: 0.09569  time: 0.2408  data_time: 0.0015  lr: 9.9902e-06  max_mem: 3807M
[10/14 00:31:08 d2.utils.events]:  eta: 0:03:40  iter: 59  total_loss: 1.006  loss_cls: 0.5046  loss_box_reg: 0.0003279  loss_rpn_cls: 0.3138  loss_rpn_loc: 0.09937  time: 0.2403  data_time: 0.0014  lr: 1.4985e-05  max_mem: 3807M
[10/14 00:31:13 d2.utils.events]:  eta: 0:03:35  iter: 79  total_loss: 0.8025  loss_cls: 0.361  loss_box_reg: 0.0002381  loss_rpn_cls: 0.3234  loss_rpn_loc: 0.09545  time: 0.2385  data_time: 0.0014  lr: 1.998e-05  max_mem: 3807M
[10/14

wandb: Waiting for W&B process to finish, PID 98632
wandb: Program ended successfully.


In [ ]:
# cfg.MODEL.WEIGHTS = "./output/model_final.pth"
# predictor = DefaultPredictor(cfg)
# cfg.MODEL.WEIGHTS = "./output/model_final.pth"
# evaluator = COCOEvaluator("test", output_dir="./output")
# val_loader = build_detection_test_loader(cfg,"test")
# metrics = inference_on_dataset(predictor.model, val_loader, evaluator)
# wandb.log(metrics)

In [ ]:
# # predictor = DefaultPredictor(cfg)
# cfg.MODEL.WEIGHTS = "./output/model_final.pth"
# evaluator = COCOEvaluator("test", output_dir="./output")
# val_loader = build_detection_test_loader(cfg,"test")
# metrics = inference_on_dataset(predictor.model, val_loader, evaluator)
# wandb.log(metrics)

In [ ]:
for img in os.listdir("./test_imgs/"):
    v = Visualizer(cv2.imread(f"./test_imgs/{img}")[:, :, ::-1], metadata=metadata)
    v = v.draw_instance_predictions(
        predictor(cv2.imread(f"./test_imgs/{img}"))["instances"].to("cpu")
    )
    v = v.get_image()[:, :, ::-1]
    plt.figure(figsize=(12, 6))
    plt.imshow(v)
    plt.savefig("./img.png")
    plt.close()

In [ ]:
# evaluator = COCOEvaluator("data", output_dir="./output")
# val_loader = build_detection_test_loader(cfg,"data")
# metrics = inference_on_dataset(predictor.model, val_loader, evaluator)
# wandb.log(metrics)
# wandb.finish()

In [ ]:
# os.environ['TENSORBOARD_BINARY'] = '/path/to/envs/my_env/bin/tensorboard'

In [ ]:
# %load_ext tensorboard

In [ ]:
# import tensorflow as tf
# import datetime, os

In [ ]:
# %tensorboard --logdir output

In [ ]:
# %tensorboard --logdir outputs # TODO : remove this this is wrong.

In [ ]:
# !ls

In [ ]:
# from tensorboard import notebook

In [ ]:
# notebook.list() # View open TensorBoard instances

In [ ]:

# from detectron2.evaluation import COCOEvaluator, inference_on_dataset
# from detectron2.data import build_detection_test_loader
# evaluator = COCOEvaluator("test", output_dir="./output")
# val_loader = build_detection_test_loader(cfg, "test")
# print(inference_on_dataset(predictor.model, val_loader, evaluator))

In [ ]:
# cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
# predictor = DefaultPredictor(cfg)

In [ ]:
# from detectron2.evaluation import COCOEvaluator, inference_on_dataset
# from detectron2.data import build_detection_test_loader
# evaluator = COCOEvaluator("data", output_dir="./output")
# val_loader = build_detection_test_loader(cfg, "data")
# print(inference_on_dataset(predictor.model, val_loader, evaluator))

In [ ]:
# cfg = get_cfg()
# cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/faster_rcnn_R_101_DC5_3x.yaml'))
# cfg.DATASETS.TRAIN = ('data',)
# cfg.DATASETS.TEST = ()
# cfg.TEST.EVAL_PERIOD = 100
# cfg.DATALOADER.NUM_WORKERS = 2
# cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
# cfg.SOLVER.IMS_PER_BATCH = 2
# cfg.SOLVER.BASE_LR = 0.00025
# cfg.SOLVER.MAX_ITER = 2500
# cfg.SOLVER.STEPS = []
# cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
# cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

In [ ]:
# predictor = DefaultPredictor(cfg)

In [ ]:
# from detectron2.evaluation import COCOEvaluator, inference_on_dataset
# from detectron2.data import build_detection_test_loader
# evaluator = COCOEvaluator("data", output_dir="./output")
# val_loader = build_detection_test_loader(cfg, "data")
# print(inference_on_dataset(predictor.model, val_loader, evaluator))